# Configuration Azure Machine Learning Service

In [13]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.65


In [31]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group,
      f'Workspace details: {ws.get_details()}', sep = '\n')

Found the config file in: C:\Users\Thijs\PycharmProjects\LEGO Vision\Notebooks\aml_config\config.json
Workspace name: LEGO-Vision
Azure region: westeurope
Subscription id: 2f5c43ac-ac34-401d-bead-2dc43ffdc4b1
Resource group: LEGO-Vision
Workspace details: {'id': '/subscriptions/2f5c43ac-ac34-401d-bead-2dc43ffdc4b1/resourceGroups/LEGO-Vision/providers/Microsoft.MachineLearningServices/workspaces/LEGO-Vision', 'name': 'LEGO-Vision', 'location': 'westeurope', 'type': 'Microsoft.MachineLearningServices/workspaces', 'description': '', 'friendlyName': '', 'containerRegistry': '/subscriptions/2f5c43ac-ac34-401d-bead-2dc43ffdc4b1/resourcegroups/lego-vision/providers/microsoft.containerregistry/registries/legovision6039404951', 'keyVault': '/subscriptions/2f5c43ac-ac34-401d-bead-2dc43ffdc4b1/resourcegroups/lego-vision/providers/microsoft.keyvault/vaults/legovision1769742005', 'applicationInsights': '/subscriptions/2f5c43ac-ac34-401d-bead-2dc43ffdc4b1/resourcegroups/lego-vision/providers/microso

In [15]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "LEGO-Visiongpu-3"

try:
    compute_target_three = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=4)

    # create the cluster
    compute_target_three = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target_three.wait_for_completion(show_output=True)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target_three.status.serialize())

Found existing compute target.


In [16]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureFile legovision1529597820 azureml-filestore-f0102bff-6d0a-47ad-b50c-4709fd2f206a


In [5]:
#ds.upload(src_dir=r'C:\Users\Thijs\Google Drive\LEGO Vision\Natuurlijke data', target_path='lego-vision-classification-natural-data', overwrite=True, show_progress=True)

$AZUREML_DATAREFERENCE_5a087d86fb764e5c8b432dcd0185f716

In [17]:
# ds_data_dir_syn = ds.path('lego-vision-classification')
ds_data_dir_nat = ds.path('lego-vision-classification-natural-data-white-background')

print(ds_data_dir_nat)

$AZUREML_DATAREFERENCE_9f525c35e3ef4fdfb33649e9b39fd773


# Support Vector Machine

## Natural Data

In [38]:
import os

project_folder = './svm-hog-dom-selfmade-hyperdrive'
os.makedirs(project_folder, exist_ok=True)

In [39]:
import shutil
shutil.copy('internal_hyperdrive_natural_data.py', project_folder)

'./svm-hog-dom-selfmade-hyperdrive\\internal_hyperdrive_natural_data.py'

In [20]:
shutil.copy('helpers.py', project_folder)

'./svm-hog-dom-selfmade-hyperdrive\\helpers.py'

In [21]:
from azureml.core import Experiment

experiment_name = 'svm-hog-dom-selfmade-hyperdrive'
experiment_natural_data = Experiment(ws, name=experiment_name)

In [43]:
from azureml.train.estimator import Estimator

script_params = {
    '--data_dir': ds_data_dir_nat,
    '--output_dir': './outputs',
    '--classifier': 'svm',
    '--number_of_samples': 3200
}

estimator_natural_data = Estimator(source_directory=project_folder, 
                                  script_params=script_params,
                                  compute_target=compute_target_three,
                                  entry_script='internal_hyperdrive_natural_data.py',
                                  use_gpu=True,
                                  conda_packages=['scikit-learn', 'matplotlib', 'Pillow', 'scikit-image'])

In [44]:
run_natural_data = experiment_natural_data.submit(estimator_natural_data)
run_natural_data

Experiment,Id,Type,Status,Details Page,Docs Page
svm-hog-dom-selfmade-hyperdrive,svm-hog-dom-selfmade-hyperdrive_1540569171924,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [41]:
from azureml.train.widgets import RunDetails
RunDetails(run_natural_data).show()

_UserRun()

In [42]:
run_natural_data.cancel()

#  Hyper drive is too unreliable

In [24]:
# from azureml.train.hyperdrive import *


# ps = GridParameterSampling(
#     {
#         '--number_of_samples': choice(200, 400, 800, 1600, 3200),
#         '--shape': choice('32', '64', '128', '256'),
#         '--color_insensitive': choice(0, 1)
#     }
# )

# early_termination_policy = BanditPolicy(slack_factor = 0.3, evaluation_interval=1, delay_evaluation=1)

# hyperdrive_run_config_natural = HyperDriveRunConfig(estimator=estimator_natural_data,
#                                                     hyperparameter_sampling=ps,
#                                                     policy=early_termination_policy,
#                                                     primary_metric_name="f_score",
#                                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
#                                                     max_total_runs=100,
#                                                     max_concurrent_runs=4)
# hyperdrive_run_config_natural

In [25]:
# hyperdrive_run_natural = experiment_natural_data.submit(hyperdrive_run_config_natural)

In [1]:
# from azureml.train.widgets import RunDetails
# RunDetails(hyperdrive_run_natural).show()

NameError: name 'hyperdrive_run_natural' is not defined

In [ ]:
# hyperdrive_run_natural.wait_for_completion(show_output=True)

RunId: svm-hog-dom_1540404252875


## Syntetic and Natural Data

In [5]:
# ds_data_dir_syn = ds.path('lego-vision-classification')
ds_data_dir_nat = ds.path('lego-vision-classification-natural-data-white-background')

print(ds_data_dir_nat)

$AZUREML_DATAREFERENCE_fed121a8712041a897c6e38d686d773d


In [6]:
# ds_data_dir_syn = ds.path('lego-vision-classification')
ds_data_dir_syn = ds.path('lego-vision-classification-synthetic-data-white-background')

print(ds_data_dir_syn)

$AZUREML_DATAREFERENCE_948c99e6abc34ad6850021f4ee59931c


In [7]:
import os

project_folder = './svm-hog-dom-synthetic-data'
os.makedirs(project_folder, exist_ok=True)

In [8]:
import shutil
shutil.copy('feature_extraction_experiment.py', project_folder)

'./svm-hog-dom-synthetic-data\\feature_extraction_experiment.py'

In [9]:
shutil.copy('helpers.py', project_folder)

'./svm-hog-dom-synthetic-data\\helpers.py'

In [10]:
from azureml.core import Experiment

experiment_name = 'svm-hog-dom-synthetic-data'
experiment_svm_synthetic_data = Experiment(ws, name=experiment_name)

In [11]:
from azureml.train.estimator import Estimator

script_params = {
    '--data_dir': ds_data_dir_syn,
    '--test_dir': ds_data_dir_nat,
    '--output_dir': './outputs',
    '--classifier': 'svm',
    '--number_of_samples': 5
}

estimator_svm_with_synthetic_data = Estimator(source_directory=project_folder, 
                                            script_params=script_params,
                                            compute_target=compute_target_seven,
                                            entry_script='feature_extraction_experiment.py',
                                            use_gpu=True,
                                            conda_packages=['scikit-learn', 'matplotlib', 'Pillow', 'scikit-image'])

In [24]:
run_svm_with_synthetic_data = experiment_svm_synthetic_data.submit(estimator_svm_with_synthetic_data)

In [25]:
from azureml.train.widgets import RunDetails
RunDetails(run_svm_with_synthetic_data).show()

_UserRun()

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


In [22]:
run_svm_with_synthetic_data.cancel()

In [12]:
from azureml.train.hyperdrive import *


ps = GridParameterSampling(
    {
        '--number_of_samples': choice(200, 400, 800, 1000),
        '--shape': choice('32', '64', '128', '256'),
        '--color_insensitive': choice(0, 1)
    }
)

early_termination_policy = BanditPolicy(slack_factor = 0.3, evaluation_interval=1, delay_evaluation=1)

hyperdrive_run_config = HyperDriveRunConfig(estimator=estimator_svm_with_synthetic_data,
                                            hyperparameter_sampling=ps,
                                            policy=early_termination_policy,
                                            primary_metric_name="f_score",
                                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs=100,
                                            max_concurrent_runs=4)

In [13]:
hyperdrive_run_synthetic_data = experiment_svm_synthetic_data.submit(hyperdrive_run_config)

The same input parameter(s) are specified in estimator script params and HyperDrive parameter space. HyperDrive parameter space definition will override duplicate entries in estimator. ['--number_of_samples'] is the list of overridden parameter(s).
